In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import time
import random
import sympy as sy
from sympy import symbols, diff, sin, lambdify, cos

# Q1)

i) Number of multiplications = n.m.k
ii) Number of multiplications = n.m.(k-1)

# Q2)

In [ ]:
n = 20  # Number of rows for the first matrix
m = 13  # Number of rows for the second matrix
k = 14  # Number of columns for both matrices

# Range of integers for the matrices
min_value = 3
max_value = 10

np.random.seed(41)
matrix_nk = np.random.randint(min_value, max_value + 1, size=(n, k))
matrix_mk = np.random.randint(min_value, max_value + 1, size=(k,m))
print(matrix_nk)
print(matrix_mk)

def matrix_multiply_without(matrix_A, matrix_B):
    if len(matrix_A[0]) != len(matrix_B):
        raise ValueError("Matrices are incompatible")

    # Initialize result matrix
    result_matrix = [[0 for _ in range(len(matrix_B[0]))] for _ in range(len(matrix_A))]

    for i in range(len(matrix_A)):
        for j in range(len(matrix_B[0])):
            for k in range(len(matrix_B)):
                result_matrix[i][j] += matrix_A[i][k] * matrix_B[k][j]

    return result_matrix

def matrix_multiply_with(matrix_A, matrix_B):
    return np.matmul(matrix_nk,matrix_mk)

In [ ]:
%timeit matrix_multiply_without(matrix_nk,matrix_mk)

In [ ]:
%timeit matrix_multiply_with(matrix_nk,matrix_mk)

The reasons for why multiplication using Numpy is much faster than using conventional loops are:

Vectorization:
NumPy operations work on entire arrays at once, using parallel computing. In contrast, when using loops in pure Python, each element in the array must be processed individually, leading to slower execution times.

C Implementation:
NumPy is implemented in C and Fortran, which are lower-level languages than Python. The core operations are executed at a lower level, resulting in faster computation compared to interpreted Python code. 

Avoiding Python Interpreter Overhead:
NumPy operations are implemented in compiled code, which avoids the interpreter overhead that comes with interpreted languages like Python. In contrast, when using pure Python loops, the interpreter needs to execute each iteration, leading to slower execution.

Broadcasting:
NumPy supports broadcasting, which allows operations between arrays of different shapes and sizes to be performed without the need for explicit looping. Broadcasting enables NumPy to efficiently handle operations on arrays of different dimensions and shapes, making code concise and faster.

# Q3) 

Using the method described in the question, the time complexity should be O(n^2). This is because to find the nth largest element, n passes of the array will be required. Since median is the (n-n//2)th largest element if n is odd or the average of (n-n//2)th and (n-(n//2)+1)th largest if n is even (n is the length of array), we'll require that many atleast n-n//2 number of passes. Since each pass iterates over the entire array, T.C = O(n^2).

A better method could be to use divide and conquer approach, which gives O(n) in average case and O(n^2) in the worst case. In this method, we choose a pivot element, and put it at its correct position, while ensuring the elements left of it are smaller and on the right are larger. Then according to the index of where the pivot element was placed, we either repeat the process on the right side or the left.
In the average case, a constant proportion of elements will fall both below and above the pivot, and thus the siz e of the array will get reduced exponentially.

For the first method, I simply used 2 nested for loops.
The second method is the quickselect algorithm.

In [ ]:
# Method 1 (of the question), to find the nth largest element
def median_pass(arr):
    n = len(arr)
    def helper(arr,n):
        if n <= 0 or n > len(arr):
            return None

        for i in range(n):
            max_element_index = i
            for j in range(i + 1, len(arr)):
                if arr[j] > arr[max_element_index]:
                    max_element_index = j

            arr[i], arr[max_element_index] = arr[max_element_index], arr[i]

        return arr[n - 1]
    
    # If length of array is odd
    if n%2 != 0:
        return helper(arr,(n-n//2))
    
    # If length of array is even
    else:
        return (helper(arr,(n-n//2))+helper(arr,((n-n//2)+1)))/2

In [ ]:
# Quickselect algorithm
def median_quickselect(arr):
    n = len(arr)
    def quick_select_kth_largest(arr,k):
        k = len(arr) - k
        def quickselect(start,end):
            p = start
            pivot = arr[end]
            for i in range(start,end):
                if arr[i] <= pivot:
                    arr[i],arr[p] = arr[p],arr[i]
                    p+=1
            arr[p],arr[end] = arr[end],arr[p]

            if p>k:
                return quickselect(start,p-1)
            elif p<k:
                return quickselect(p+1,end)
            else:
                return arr[p]
        return quickselect(0,len(arr)-1)
    
    if n%2 != 0:
        return quick_select_kth_largest(arr,(n-n//2))
    else:
        return (quick_select_kth_largest(arr,(n-n//2))+quick_select_kth_largest(arr,((n-n//2)+1)))/2
    
test_arr = [random.randint(0, 1000) for _ in range(1000)]

In [ ]:
%timeit median_pass(test_arr)

In [ ]:
%timeit median_quickselect(test_arr)

In [ ]:
%timeit np.median(test_arr)

Clearly, np.median is around a thousand times faster than the other 2 methods. The quickselect algorithm probably is taking more time to run is because of the recursion calls involved. Even if we sort the array and access the middle element, it is taking a longer time than the O(N^2) approach.

# Q4)

In [ ]:
np.random.seed(30)

# Taking uniform gives us float values, which is required for jax.grad
a = np.random.uniform(0, 10, 3)
b = np.random.uniform(0, 10, 3)

In [ ]:
#Manually

x, y = symbols('x y')

f_x = (2*x*y) + (y**3) * cos(x)

df_dx_fun = lambdify([x, y], f_x, 'numpy')

f_y = (x**2) + (3*y**2) * sin(x)
df_dy_fun = lambdify([x, y], f_y, 'numpy')

res_dx = df_dx_fun(a, b)
res_dy = df_dy_fun(a, b)

print(f"Result of df/dx at x={a} and y={b}: {res_dx}")
print(f"Result of df/dy at x={a} and y={b}: {res_dy}")

In [ ]:
# Simpy
x, y = symbols('x y')
f = x**2 * y + y**3 * sin(x)

df_dx = diff(f, x)
df_dy = diff(f, y)

df_dx_func = lambdify([x, y], df_dx, 'numpy')
df_dy_func = lambdify([x, y], df_dy, 'numpy')

result_dx = df_dx_func(a, b)
result_dy = df_dy_func(a, b)

print(f"Result of df/dx at x={a} and y={b}: {result_dx}")
print(f"Result of df/dy at x={a} and y={b}: {result_dy}")

In [ ]:
# JAX

import jax
import jax.numpy as jnp
import numpy as np
from sympy import sin,cos

def f(x, y):
    return (x*2)(y) + (y**3)*jnp.sin(x)

def analytical_gradient(x, y):
    df_dx = 2*x*y + (y**3)*jnp.cos(x)
    df_dy = (x*2) + 3(y**2)*jnp.sin(x)
    return np.array([df_dx, df_dy])

grad_fun = jax.grad(f,argnums=[0,1])

computed_gradient = []
for i in range(len(a)):
    computed_gradient.append(grad_fun(a[i],b[i]))

print("Computed Gradient:", computed_gradient)

# Q7)

In [ ]:
academic_data = {
    2022: {
        'Branch 1': [
            {'Roll Number': 1, 'Name': 'A', 'Marks': {'Maths': 100, 'English': 70}},
            {'Roll Number': 2, 'Name': 'B', 'Marks': {'Maths': 70, 'English': 100}},
            {'Roll Number': 3, 'Name': 'C', 'Marks': {'Maths': 70, 'English': 70}}
            
        ],
        'Branch 2': [
        ]
    },
    2023: {
        'Branch 1': [
        ],
        'Branch 2': [
        ]
    },
    2024: {
        'Branch 1': [
        ],
        'Branch 2': [
        ]
    },
    2025: {
        'Branch 1': [
        ],
        'Branch 2': [
        ]
    }
}

# Accessing the data for the first roll number in 'branch 1' in A.Y. 2022
print(f'Name of the first roll no. ( = {academic_data[2022]["Branch 1"][0]["Roll Number"]}) in "Branch 1" in A.Y. 2022 was: {academic_data[2022]["Branch 1"][0]["Name"]}')
print(f'Marks in Maths and English of the first roll no.( = {academic_data[2022]["Branch 1"][0]["Roll Number"]}) in "Branch 1" in A.Y. 2022 were {academic_data[2022]["Branch 1"][0]["Marks"]["Maths"]} and {academic_data[2022]["Branch 1"][0]["Marks"]["English"]} respectively\n')
print(academic_data)

# Q8)

In [ ]:
class Student:
    def __init__(self, roll_number, name, marks):
        self.roll_number = roll_number
        self.name = name
        self.marks = marks

    def __str__(self):
        return f"Roll Number: {self.roll_number}, Name: {self.name}, Marks: {self.marks}"


class Branch:
    def __init__(self, name):
        self.name = name
        self.students = []

    def add_student(self, student):
        self.students.append(student)

    def __str__(self):
        return f"Branch: {self.name}, Students: {self.students}"


class Year:
    def __init__(self, year):
        self.year = year
        self.branches = []

    def add_branch(self, branch):
        self.branches.append(branch)

    def __str__(self):
        return f"Year: {self.year}, Branches: {self.branches}"


# Overall Database (List of Year objects)
database = []

# Creating students
student1 = Student(1, 'A', {'Maths': 100, 'English': 70})
student2 = Student(2, 'B', {'Maths': 70, 'English': 100})
student3 = Student(3, 'C', {'Maths': 70, 'English': 70})

# Creating branches and adding students
branch1 = Branch('Branch 1')
branch1.add_student(student1)
branch1.add_student(student2)
branch1.add_student(student3)

branch2 = Branch('Branch 2')

# Creating years and adding branches
year2022 = Year(2022)
year2022.add_branch(branch1)
year2022.add_branch(branch2)

year2023 = Year(2023)

year2024 = Year(2024)

year2025 = Year(2025)

# Adding years to the database
database.append(year2022)
database.append(year2023)
database.append(year2024)
database.append(year2025)

print(f'Name of the first roll no. ( = {database[0].branches[0].students[0].roll_number}) in "Branch 1" in A.Y. 2022 was: {database[0].branches[0].students[0].name}')

# Q9)

In [ ]:
# Domain and step size
x_values = np.arange(0.5, 100.5, 0.5)

fig, ax = plt.subplots(7,figsize=(8, 16))
ax[0].plot(x_values, x_values)
ax[1].plot(x_values, (x_values)**2)
ax[2].plot(x_values, ((x_values)**3)/100)
ax[3].plot(x_values, np.sin(x_values))
ax[4].plot(x_values, (np.sin(x_values))/(x_values))
ax[5].plot(x_values, np.log(x_values))
ax[6].plot(x_values, np.exp(x_values))

ax[0].set_title('y = x')
ax[1].set_title('y = x^2')
ax[2].set_title('y = (x^3)/100')
ax[3].set_title('y = sin(x)')
ax[4].set_title('y = sin(x)/x')
ax[5].set_title('y = log(x)')
ax[6].set_title('y = e^x')

for i in range(7):
    ax[i].set_xlabel('x')
    ax[i].set_ylabel('y')


plt.subplots_adjust(hspace=1.5)


plt.show()

# Q10)

In [ ]:
np.random.seed(0) 
matrix = np.random.uniform(1, 2, size=(20, 5)) # Generating a random 20*5 matrix
df = pd.DataFrame(matrix, columns=['a', 'b', 'c', 'd', 'e']) # Creating a dataframe from the matrix
max_std_column = df.std(axis = 0).idxmax()
print(f"Column with the highest standard deviation: {max_std_column}")
min_mean_row = df.mean(axis = 1).idxmin()
print(f"Row with the lowest mean: {min_mean_row}")
df

# Q11)

In [ ]:
df['f'] = df[['a', 'b', 'c', 'd', 'e']].sum(axis=1) # Adding 'f' column
df['g'] = np.where(df['f'] < 8, 'LT8', 'GT8') # Adding 'g' column

num_rows_lt8 = len(df[df['g'] == 'LT8'])
print(f'No. of rows in which value in "f" column is < 8: {num_rows_lt8}')

std_lt8 = df.loc[df['g'] == 'LT8', 'f'].std() #standard deviation of column "f" for rows where "g" is "LT8"
std_gt8 = df.loc[df['g'] == 'GT8', 'f'].std() #standard deviation of column "f" for rows where "g" is "GT8"

print("Standard deviation of 'f' for rows where 'g' is 'LT8':", std_lt8)
print("Standard deviation of 'f' for rows where 'g' is 'GT8':", std_gt8)
df

# Q12)

Broadcasting is a feature that enables arrays with different shapes to be used in arithmetic operations. The smaller array is broadcasted across the larger array so that they have compatible shapes, or both the arrays can be broadcast as well. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It can be thought of as stretching the incompatible values, without actually utilizing any additional memory.

In [ ]:
# Eg 1
a = np.array([1,2,3])
b = 5
print(f'Sum of a and b is = {a+b}')  # Scalar b is 'stretched' to (3,1) to enable vectorisation and element-wise addition

# Eg 2
c = np.arange(3)
d = np.arange(3)[:, np.newaxis]

print(f'Shape of matrix c is {c.shape}')
print(f'Shape of matrix d is {d.shape}')
print(f'Sum of matrix c and d is = {c+d}') # c and d are both broadcast to a common shape, i.e (3,3)

# Q13)

In [ ]:
# Will return the index of the first occurence of the minimum element. If array is multidimensional, it will flatten it and treat it as a 1-D array

def solve(arr,min_val =sys.maxsize,count = -1,index = -1):
    if arr.ndim == 1:
        for element in arr:
            index += 1
            if element < min_val:
                count = index
                min_val = element
                
    else:
        for subarray in arr:
            min_val,count,index = solve(subarray,min_val,count,index) 
    return [min_val,count,index]

def argmin_personal(arr):
    return solve(arr)[1]

matrix_test = np.random.rand(2, 3, 2)

print(f'Minimum element occurs in the numpy array for the first time at index = {argmin_personal(matrix_test)}')
print(f'Minimum element occurs in the numpy array for the first time at index by built-in function = {np.argmin(matrix_test)}')
print(f'Answer obtained is same as that of "np.argmin"? Yes' if np.argmin(matrix_test) == argmin_personal(matrix_test) else 'No')